In [5]:
import numpy as np
import pulp

# Define the problem
my_obj = [1, 2, 3, 1]
my_ub = [40, None, None, 3]
my_lb = [0, 0, 0, 2]
my_ctype = ["Continuous", "Continuous", "Continuous", "Integer"]
my_colnames = ["x1", "x2", "x3", "x4"]
my_rhs = [20, 30, 0]
my_rownames = ["c1", "c2", "c3"]
my_sense = ["<=", "<=", "="]

# Create the problem
prob = pulp.LpProblem("Maximize_Objective", pulp.LpMaximize)

# Create decision variables
variables = []
for i, name in enumerate(my_colnames):
    var_type = pulp.LpContinuous if my_ctype[i] == "Continuous" else pulp.LpInteger
    var = pulp.LpVariable(name, lowBound=my_lb[i], upBound=my_ub[i], cat=var_type)
    variables.append(var)

# Set the objective function
prob += pulp.lpDot(my_obj, variables), "Objective"

# Add constraints
constraints = [
    ([-1, 1, 1, 10], "<=", 20),
    ([1, -1, 1, 0], "<=", 30),
    ([0, 1, 0, -3.5], "=", 0)
]

for i, (coeffs, sense, rhs) in enumerate(constraints):
    expr = pulp.lpDot(coeffs, variables)
    if sense == "<=":
        prob += (expr <= rhs), my_rownames[i]
    elif sense == ">=":
        prob += (expr >= rhs), my_rownames[i]
    elif sense == "=":
        prob += (expr == rhs), my_rownames[i]

# Solve the problem using CPLEX solver
prob.solve(pulp.CPLEX_CMD(msg=0))

# Print the solution status
print("Solution status =", pulp.LpStatus[prob.status])

# Print the objective value
print("Objective value =", pulp.value(prob.objective))

# Print variable values
for v in variables:
    print(f"{v.name} =", v.varValue)  # 获取变量的最终取值


PulpSolverError: PuLP: Error while trying to execute cplex.exe